In [1]:
import numpy as np
import math

### Here you will find the Newton method to find the 'D' component for coding

In [13]:
def dx(f, x):
    return abs(0-f(x))
 
def newtons_method(f, df, guess, e):
    delta = dx(f, guess)
    while delta > e:
        guess = guess - f(guess)/df(guess)
        delta = dx(f, guess)
    print ('D = ', guess)

In [16]:
Pb = 1e-8
def compute_D():
    
    '''Here we use the 3 terms approximations.
    Il faut faire attention : Le 1 term approximation n'est valable SEULEMENT SI la proba d'erreur est basse.
    Donc pour Pb ~ 10^-5 il faut commencer à prendre au moins 2 termes.

    Ici avec 3 termes on est laaarge.'''
    
    
    def f(D):
        global Pb
        return 0.5*(36*D**10 + 211*D**12 + 1404*D**14) - Pb
 
    def df(D):
        return 0.5*(360*D**9 + 12*211*D**11 + 14*1404*D**13)
    
    def dx(f, x):
        return abs(0-f(x))

    def newtons_method(f, df, guess, e):
        delta = dx(f, guess)
        while delta > e:
            guess = guess - f(guess)/df(guess)
            delta = dx(f, guess)
        print ('D = ', guess)
        
    
    basic_guess = 0.2
    eps = 1e-15;
    
    return newtons_method(f, df, basic_guess, eps)
    

In [17]:
compute_D()

D =  0.11770166447765293
